# Librerías

os: Para manejar directorios y archivos.

cv2: Librería OpenCV para procesamiento de imágenes.

numpy: Para cálculos matemáticos y operaciones con matrices.

In [ ]:
import os
import cv2
import numpy as np

# Funciones de Procesamiento de Imágenes

Implementé algunas funciones para tratar las imagenes, por lo que de una imagen se generan varias versiones, se guarda la imagen original y su versión con cada filtro.

La primera de ellas es para ajustar el brillo y contraste, consideré que sería bueno implementar esta función ya que podría ayudar a mejorar la generalización del modelo al exponerlo a imágenes con diferentes niveles de iluminación.

In [ ]:
def ajustar_brillo_contraste(frame, alpha=1.0, beta=0):
    return cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)

Otra función es el recorte aleatorio, esto con la finalidad de extraer características específicas del coche, tomando en cuenta que en la imagenes podría salir solamente una parte del coche.

In [ ]:
def recortar_aleatoriamente(frame, porcentaje=0.1):
    h, w = frame.shape[:2]
    dy, dx = int(h * porcentaje), int(w * porcentaje)
    x1, y1 = np.random.randint(0, dx), np.random.randint(0, dy)
    return frame[y1:h - dy + y1, x1:w - dx + x1]

Esta función de zoom es para simular diferentes distancias 

In [ ]:
def aplicar_zoom(frame, factor=1.2):
    h, w = frame.shape[:2]
    nh, nw = int(h / factor), int(w / factor)
    frame_zoom = frame[(h - nh) // 2:(h + nh) // 2, (w - nw) // 2:(w + nw) // 2]
    return cv2.resize(frame_zoom, (w, h))

Esta versión de la imagen es para eliminar el ruido y que la imagen quede más clara.

In [ ]:
def eliminar_ruido(frame):
    return cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)

Aquí se aplican los filtros blanco y negro, para ayudar en la parte de la detección de bordes y formas, rgb para ayudar a la clasificación de coches de acuerdo a los patrones de color y el filtro de escala de grises para el análisis de patrones y texturas.

In [ ]:
def aplicar_filtros_basicos(frame, filtro):
    if filtro == "blanco_negro":
        return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif filtro == "rgb":
        return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    elif filtro == "escala_grises":
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    else:
        return frame

Aquí se rota la imagen en un rango de ángulos aleatorios y se redimensiona, además se aumenta la variación en las posiciones de los coches en las imágenes.

In [ ]:
def rotar_y_redimensionar(frame, angulo_max=30, tamaño=(128, 128)):
    angulo = np.random.uniform(-angulo_max, angulo_max)
    
    h, w = frame.shape[:2]
    centro = (w // 2, h // 2)
    
    matriz_rotacion = cv2.getRotationMatrix2D(centro, angulo, 1)
    
    imagen_rotada = cv2.warpAffine(frame, matriz_rotacion, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    
    imagen_redimensionada = cv2.resize(imagen_rotada, tamaño)
    
    return imagen_redimensionada

Se aplica rotaciones predefinidas de 15, 30, 45, 60 y 90 grados, útil para garantizar variaciones consistentes en el dataset.

In [ ]:
def rotar_varias_veces(frame, angulos=[15, 30, 45, 60, 90], tamaño=(128, 128)):
    rotaciones = []
    for angulo in angulos:
        rotacion = rotar_y_redimensionar(frame, angulo_max=angulo, tamaño=tamaño)
        rotaciones.append(rotacion)
    return rotaciones

Ajusta el tono y la saturación en el espacio de color HSV y simula condiciones de color ambiental (amanecer, atardecer, etc.).


In [ ]:
def cambiar_tono_saturacion(frame, alpha=1.2, beta=50):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv[..., 0] = hsv[..., 0] * alpha + beta 
    hsv[..., 1] = hsv[..., 1] * alpha         
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

Añade ruido de tipo "sal y pimienta" lo que simula imágenes con interferencias.

El ruido de sal y pimienta es un tipo de ruido que se presenta en imágenes digitales, y se caracteriza por la aparición de píxeles blancos y negros dispersos por la imagen. Este efecto es similar al de esparcir sal y pimienta sobre la imagen.


In [ ]:
def agregar_ruido_sal_pimienta(frame, cantidad=0.02):
    h, w = frame.shape[:2]
    salida = np.copy(frame)
    num_pixeles = int(cantidad * h * w)
    
    for _ in range(num_pixeles):
        x, y = np.random.randint(0, w), np.random.randint(0, h)
        salida[y, x] = np.random.choice([0, 255], size=3)
    
    return salida

Aplica un desenfoque gaussiano para suavizar la imagen es útil para simular condiciones de enfoque o cámaras de baja calidad.

In [ ]:
def desenfoque_gaussiano(frame, kernel_size=(5, 5)):
    return cv2.GaussianBlur(frame, kernel_size, 0)

Aquí se aplica una escla de color y luego se realiza la versión espejo de cada imagen.

In [ ]:
def escala_color(frame, factor=1.0):
    return cv2.convertScaleAbs(frame, alpha=factor, beta=0)

def version_espejo(frame):
    return cv2.flip(frame, 1)

Aqupi se mandan a llamar las funciones de los filtros para crear las versiones de la imagen (también se aplica la versión espejo de cada versión de la imagen con disitinto filtro) y guardarlas en formato jpg.

In [ ]:
def generar_versiones(output_folder, base_name, frame):
    transformaciones = {
        "rotacion_volteo": lambda img: cv2.flip(cv2.rotate(img, cv2.ROTATE_180), 1),
        "brillo_contraste": lambda img: ajustar_brillo_contraste(img, alpha=1.5, beta=30),
        "desplazamiento_zoom": lambda img: aplicar_zoom(img, factor=1.3),
        "recorte": lambda img: recortar_aleatoriamente(img, porcentaje=0.2),
        "eliminacion_ruido": lambda img: eliminar_ruido(img),
        "filtro_bn": lambda img: aplicar_filtros_basicos(img, "blanco_negro"),
        "filtro_rgb": lambda img: aplicar_filtros_basicos(img, "rgb"),
        "filtro_grises": lambda img: aplicar_filtros_basicos(img, "escala_grises"),
        "brillo_alto": lambda img: ajustar_brillo_contraste(img, alpha=2.0, beta=50),
        "brillo_medio": lambda img: ajustar_brillo_contraste(img, alpha=1.5, beta=25),
        "brillo_bajo": lambda img: ajustar_brillo_contraste(img, alpha=0.8, beta=-30),
        "contraste_alto": lambda img: ajustar_brillo_contraste(img, alpha=2.5, beta=0),
        "contraste_bajo": lambda img: ajustar_brillo_contraste(img, alpha=0.5, beta=0),
        "rotacion_aleatoria": lambda img: rotar_y_redimensionar(img),
        "rotaciones_adicionales": lambda img: rotar_varias_veces(img),
        "cambiar_tono_saturacion": lambda img: cambiar_tono_saturacion(img),
        "ruido_sal_pimienta": lambda img: agregar_ruido_sal_pimienta(img),
        "desenfoque_gaussiano": lambda img: desenfoque_gaussiano(img),
        "escala_color": lambda img: escala_color(img),
        "espejo": lambda img: version_espejo(img) 
    }

    for nombre, transformacion in transformaciones.items():
        img_transformada = transformacion(frame)
        if isinstance(img_transformada, list): 
            for i, img in enumerate(img_transformada):
                cv2.imwrite(f"{output_folder}/{base_name}_{nombre}_{i}.jpg", img)
        else:
            cv2.imwrite(f"{output_folder}/{base_name}_{nombre}.jpg", img_transformada)

    frame_espejo = version_espejo(frame)

    for nombre, transformacion in transformaciones.items():
        img_transformada = transformacion(frame_espejo)
        if isinstance(img_transformada, list):  
            for i, img in enumerate(img_transformada):
                cv2.imwrite(f"{output_folder}/{base_name}_{nombre}_espejo_{i}.jpg", img)
        else:
            cv2.imwrite(f"{output_folder}/{base_name}_{nombre}_espejo.jpg", img_transformada)

Aqupi, creamos una carpeta para guardar el dataset en caso de que no exista y se leen las imagenes de una carpeta para posteriormente aplicarles los filtros anteriores-

In [ ]:
def generar_dataset_imagenes(input_folder, output_folder, resolution=(128, 128)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        if not os.path.isfile(img_path):
            continue

        frame = cv2.imread(img_path)
        if frame is None:
            print(f"No se pudo leer la imagen: {img_name}")
            continue

        frame_resized = cv2.resize(frame, resolution)

        base_name, _ = os.path.splitext(img_name)
        cv2.imwrite(f"{output_folder}/{base_name}_original.jpg", frame_resized)

        generar_versiones(output_folder, base_name, frame_resized)
        print(f"Procesada la imagen: {img_name}")

    print(f"Dataset generado en {output_folder}")

Por último, colocamos la ruta de la carpeta que contiene las imagenes a tratar, así como la carpeta donde se guarda el dataset.

In [ ]:
input_folder = "C:/Users/ShiEu/Documents/9 Semestre/imagenes"
output_folder = "C:/Users/ShiEu/Documents/dataset_coches"

generar_dataset_imagenes(input_folder, output_folder)